In [20]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pyfits
from scipy import stats
from scipy import interpolate
import itertools
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
print(cosmo)

FlatLambdaCDM(H0=70 km / (Mpc s), Om0=0.3, Tcmb0=2.725 K, Neff=3.04, m_nu=[ 0.  0.  0.] eV, Ob0=None)


In [21]:
z_f = float(2.0)   #redshift of the fiducial field
z2 = float(0.348)    #redshift of the lens
Pix_ratio = 3173304.711
Npix=3584
Npix0=512

In [22]:
data_dir='/home/jess/RXJ2248/WSLAP/mag/mag_data.txt'
pick_dir='/home/jess/RXJ2248/WSLAP/mag/idl_plot/pick'
exclude_dir='/home/jess/RXJ2248/WSLAP/mag/idl_plot/exclude'
exclude_sys_dir='/home/jess/RXJ2248/WSLAP/mag/idl_plot/exclude_sys'
alphaX_512_rad_dir = '/home/jess/RXJ2248/WSLAP/mag/recomp_alpha_x_rad.fits'
alphaY_512_rad_dir = '/home/jess/RXJ2248/WSLAP/mag/recomp_alpha_y_rad.fits'

In [23]:

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Read fits files
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

###read the alphax file
hdulist = pyfits.open(alphaX_512_rad_dir)
alphaX_fid_512_rad = (hdulist[0].data)   #alphaX in 512*512, radian, fiducial field


###read the alphaY file
hdulist = pyfits.open(alphaY_512_rad_dir)
alphaY_fid_512_rad = (hdulist[0].data)   #alphaY in 512*512, radianm fiducial field
print(alphaY_fid_512_rad*Pix_ratio)
print(alphaY_fid_512_rad[239,191]*Pix_ratio)

[[-245.65930176 -246.48370361 -247.30863953 ..., -299.3210144  -297.90914917
  -296.49768066]
 [-245.6388855  -246.46502686 -247.29174805 ..., -299.10073853
  -297.68606567 -296.27178955]
 [-245.61251831 -246.44036865 -247.26879883 ..., -298.86056519
  -297.44332886 -296.02655029]
 ..., 
 [ 298.77441406  300.26904297  301.76501465 ...,  242.75596619
   241.92324829  241.09152222]
 [ 299.09939575  300.59179688  302.08554077 ...,  242.80738831
   241.97637939  241.14634705]
 [ 299.40350342  300.89343262  302.38461304 ...,  242.85328674
   242.02403259  241.19573975]]
-271.954884821


In [24]:
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
# Interpolate alpha
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
print(' Interpolating alpha ... ')
#alpha_X
x = np.arange(Npix0)
y = np.arange(Npix0)
fX = interpolate.interp2d(x, y, alphaX_fid_512_rad)

ratio = float(Npix0)/float(Npix)
xnew = np.arange(0,Npix0,ratio)
ynew = np.arange(0,Npix0,ratio)
alphaX_fid_ACS_rad = fX(xnew, ynew)

print('alphaX has dimension:' + str(alphaX_fid_ACS_rad.shape))

#alpha_Y
fY = interpolate.interp2d(x, y, alphaY_fid_512_rad)
alphaY_fid_ACS_rad = fY(xnew, ynew)

print('alphaY has dimension:' + str(alphaY_fid_ACS_rad.shape))

print('Interpolation Done')

alphaX_fid_ACS = alphaX_fid_ACS_rad*Pix_ratio   #alphaX in 
alphaY_fid_ACS = alphaY_fid_ACS_rad*Pix_ratio


 Interpolating alpha ... 
alphaX has dimension:(3584, 3584)
alphaY has dimension:(3584, 3584)
Interpolation Done


In [35]:
a=[[4.2,5.5,3.2],[2.3,2.4,2.5], [3.6,2.5,1.3]]
a=np.array(a)
b=[1.1,2.2,3.3]
a=np.insert(a,0,b,axis=1)
a=np.insert(a,4,b,axis=1)
print(a)
#print(str(a.shape))
c=np.ndarray.transpose(np.array([(a[:,x+1]-a[:,x-1])/2 for x in range(1,len(a[0,:])-1)]))
c_0=np.array([a[:,1]-a[:,0]])
c_end=np.array([a[:,len(a[0,:])-1]-a[:,len(a[0,:])-2]])
c=np.insert(c,0,c_0,axis=1)
c=np.insert(c,len(a[0,:])-1,c_end,axis=1)
print(c_end)
print(len(a[0,:])-1)
print(c)

c=np.array([(a[x+1,:]-a[x-1,:])/2 for x in range(1,len(a[:,0])-1)])
c_0=np.array([a[1,:]-a[0,:]])
c_end=np.array([a[len(a[:,0])-1,:]-a[len(a[:,0])-2,:]])
c=np.insert(c,0,c_0,axis=0)
c=np.insert(c,len(a[:,0])-1,c_end,axis=0)
print(c_end)
print(len(a[:,0])-1)
print(len(c[0,:]))
print(c)


[[ 1.1  4.2  5.5  3.2  1.1]
 [ 2.2  2.3  2.4  2.5  2.2]
 [ 3.3  3.6  2.5  1.3  3.3]]
[[-2.1 -0.3  2. ]]
4
[[ 3.1   2.2  -0.5  -2.2  -2.1 ]
 [ 0.1   0.1   0.1  -0.1  -0.3 ]
 [ 0.3  -0.4  -1.15  0.4   2.  ]]
[[ 1.1  1.3  0.1 -1.2  1.1]]
2
5
[[ 1.1  -1.9  -3.1  -0.7   1.1 ]
 [ 1.1  -0.3  -1.5  -0.95  1.1 ]
 [ 1.1   1.3   0.1  -1.2   1.1 ]]


In [32]:
a_xx_mid=np.array([(alphaX_fid_ACS[:,x+1]-alphaX_fid_ACS[:,x-1])/2 for x in range(1,Npix-1)])
print(a_xx_mid)
a_xx_mid=np.array(a_xx_mid)
a_xx_mid=np.ndarray.transpose(a_xx_mid)
#print(a_xx_mid)
#print('a_xx_mid has dimension:', str(a_xx_mid.shape))
a_xx_0=np.array(([alphaX_fid_ACS[:,1]-alphaX_fid_ACS[:,0]]))
a_xx_end=np.array(([alphaX_fid_ACS[:,Npix-1]-alphaX_fid_ACS[:,Npix-2]]))
a_xx=np.insert(a_xx_mid,0, a_xx_0, axis=1)
a_xx=np.insert(a_xx,Npix-1, a_xx_end, axis=1)
print('a_xx has dimension:', str(a_xx.shape))
#print(a_xx)

a_yx_mid=np.array([(alphaY_fid_ACS[:,x+1]-alphaY_fid_ACS[:,x-1])/2 for x in range(1,Npix-1)])
a_yx_mid=np.array(a_yx_mid)
a_yx_mid=np.ndarray.transpose(a_yx_mid)
a_yx_0=np.array(([alphaY_fid_ACS[:,1]-alphaY_fid_ACS[:,0]]))
a_yx_end=np.array(([alphaY_fid_ACS[:,Npix-1]-alphaY_fid_ACS[:,Npix-2]]))
a_yx=np.insert(a_yx_mid,0, a_yx_0, axis=1)
a_yx=np.insert(a_yx,Npix-1, a_yx_end, axis=1)
print('a_yx has dimension:', str(a_yx.shape))
#print(a_yx)

a_yy_mid=np.array([(alphaY_fid_ACS[x+1,:]-alphaY_fid_ACS[x-1,:])/2 for x in range(1,Npix-1)])
a_yy_0=np.array(([alphaY_fid_ACS[1,:]-alphaY_fid_ACS[0,:]]))
a_yy_end=np.array(([alphaY_fid_ACS[Npix-1,:]-alphaY_fid_ACS[Npix-2,:]]))
a_yy=np.insert(a_yy_mid,0, a_yy_0, axis=0)
a_yy=np.insert(a_yy,Npix-1, a_yy_end, axis=0)
print('a_yy has dimension:', str(a_yy.shape))
#print(a_yy)

a_xy_mid=np.array([(alphaX_fid_ACS[x+1,:]-alphaX_fid_ACS[x-1,:])/2 for x in range(1,Npix-1)])
a_xy_0=np.array(([alphaX_fid_ACS[1,:]-alphaX_fid_ACS[0,:]]))
a_xy_end=np.array(([alphaX_fid_ACS[Npix-1,:]-alphaX_fid_ACS[Npix-2,:]]))
a_xy=np.insert(a_xy_mid,0, a_xy_0, axis=0)
a_xy=np.insert(a_xy,Npix-1, a_xy_end, axis=0)
print('a_xy has dimension:', str(a_xy.shape))

[[ 0.0256418   0.02562907  0.02561635 ...,  0.05803213  0.05803213
   0.05803213]
 [ 0.0256418   0.02562907  0.02561635 ...,  0.05803213  0.05803213
   0.05803213]
 [ 0.0256418   0.02562907  0.02561635 ...,  0.05803213  0.05803213
   0.05803213]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
('a_xx has dimension:', '(3584, 3584)')
('a_yx has dimension:', '(3584, 3584)')
('a_yy has dimension:', '(3584, 3584)')
('a_xy has dimension:', '(3584, 3584)')


In [33]:
f1 = open(data_dir, 'r')
data=[]
for row in f1.readlines():
    if not row.startswith('#'):
        t=row.strip().split("\t")
        t1 = [float(x) for x in t]
        data.append(t1)
data=np.array(data)
print(data[:,0])

[  2.1   2.2   2.3   3.1   3.2   3.3   4.1   4.2   4.3   6.1   6.2   6.3
   7.1   7.2   7.3   8.1   8.2   8.3  11.1  11.2  11.3  12.1  12.2  12.3
  12.4  12.5  18.1  18.2  20.1  20.2  20.3]


In [34]:
f2 = open(pick_dir, 'r')
pick=[]
for row in f2.readlines():
        t=row.strip()
        t1 = int(t)
        pick.append(t1)
pick=np.array(pick)
print (pick)   

f3 = open(exclude_dir, 'r')
exclude=[]
for row in f3.readlines():
        t=row.strip()
        t1=round(float(t),1)
        exclude.append(t1)
exclude=np.array(exclude)
print (exclude) 

f4 = open(exclude_sys_dir, 'r')
exclude_sys=[]
for row in f4.readlines():
        t=row.strip()
        t1=int(t)
        exclude_sys.append(t1)
exclude_sys=np.array(exclude_sys)
#exclude_sys=[]
print (exclude_sys) 

[ 2  3  4  6  7  8 11 12 20]
[]
[2 3]


In [35]:
data=data[np.array([round(data[x,0],0) not in exclude_sys for x in range(len(data[:,0]))]),:]
#print(data)
#print(len(data))
#print(data[:,0])

data=data[np.logical_not(np.logical_or(data[:,4] == -99, data[:,5] == -99)),:]
#print(data)
print(len(data))
print(data[:,0])

22
[  4.2   4.3   6.1   6.2   6.3   7.1   7.3   8.1   8.2   8.3  11.1  11.2
  11.3  12.1  12.2  12.3  12.4  18.1  18.2  20.1  20.2  20.3]


In [36]:
z_src=data[:,3]
DlsDs=[cosmo.angular_diameter_distance_z1z2(z2,z_src[x])/cosmo.angular_diameter_distance(z_src[x]) for x in range(len(z_src))]
DlfDf=cosmo.angular_diameter_distance_z1z2(z2,z_f)/cosmo.angular_diameter_distance(z_f)
f_k=[DlsDs[x]/DlfDf for x in range(len(z_src))]
f_k=np.array(f_k)
print(f_k)
datas=np.insert(data,len(data[0,:]), f_k, axis=1)
print(datas[:,len(data[0,:])-1])

[ 0.91311395  0.91311395  0.9192253   0.9192253   0.9192253   0.81014556
  0.81014556  0.98221456  0.98221456  0.98221456  1.07135064  1.07135064
  1.07135064  1.13475709  1.13475709  1.13475709  1.13475709  1.10240981
  1.10240981  1.07143255  1.07143255  1.07143255]
[ 0.04  0.03  0.01  0.02  0.01  0.04  0.04  0.01  0.04  0.04  0.08  0.06
  0.2   0.08  0.24  0.07  0.6   0.26  0.23  0.07  0.07  0.13]


In [37]:
centroid_x=np.round(np.array(data[:,1])).astype(int)
centroid_y=np.round(np.array(data[:,2])).astype(int)
print(centroid_y)

[1992 2198 1290 1702 1936 1376 1830 1261 1448 1995 1232 1289 1868 1429 1586
 1840 2406 1324 1360 1361 1517 2280]


In [38]:
axx=[f_k[x]*a_xx[centroid_y[x], centroid_x[x]] for x in range(len(centroid_x))]
ayx=[f_k[x]*a_yx[centroid_y[x], centroid_x[x]] for x in range(len(centroid_x))]
axy=[f_k[x]*a_xy[centroid_y[x], centroid_x[x]] for x in range(len(centroid_x))]
ayy=[f_k[x]*a_yy[centroid_y[x], centroid_x[x]] for x in range(len(centroid_x))]

M=[abs(1/((1-axx[x])*(1-ayy[x])-(ayx[x]*axy[x]))) for x in range(len(f_k))]
print(M)
datat=np.insert(datas,len(datas[0,:]), M, axis=1)
data=datat
print(data[:,len(data[0,:])-1])

[3.0214232099072316, 5.8730550617469568, 3.2180788916449177, 15.593769821413996, 10.040629448247017, 5.7148044442924819, 6.3057349663540441, 5.9507266293074927, 4.9106907508857702, 3.5541645563736726, 18.023072107108916, 8.3037826914267825, 3.7528710907157921, 5.6704482919996799, 9.7474092121946949, 2.6492113038112639, 1.8164868133589505, 22.711956581804564, 83.967917806531815, 6.895429271894022, 4.1331619797522352, 2.1398050259751247]
[  3.02142321   5.87305506   3.21807889  15.59376982  10.04062945
   5.71480444   6.30573497   5.95072663   4.91069075   3.55416456
  18.02307211   8.30378269   3.75287109   5.67044829   9.74740921
   2.6492113    1.81648681  22.71195658  83.96791781   6.89542927
   4.13316198   2.13980503]


In [39]:
data1=[]
data1=data[np.array([round(data[x,0],0) in pick for x in range(len(data[:,0]))]),:]
#print(data1[:,0])
#print(len(data1[:,0]))

data1=data1[np.array([data1[x,0] not in exclude for x in range(len(data1[:,0]))]),:]
data1=data1[np.array([data1[x,1] not in exclude for x in range(len(data1[:,1]))]),:]
print(data1[:,7])
print(data1[:,0])
print(len(data1[:,0]))

data2=[]
data2=data[np.array([data[x,0] not in data1[:,0] for x in range(len(data[:,0]))]),:]
print(data2)
print(data2[:,0])
print(len(data2[:,0]))

[  3.02142321   5.87305506   3.21807889  15.59376982  10.04062945
   5.71480444   6.30573497   5.95072663   4.91069075   3.55416456
  18.02307211   8.30378269   3.75287109   5.67044829   9.74740921
   2.6492113    1.81648681   6.89542927   4.13316198   2.13980503]
[  4.2   4.3   6.1   6.2   6.3   7.1   7.3   8.1   8.2   8.3  11.1  11.2
  11.3  12.1  12.2  12.3  12.4  20.1  20.2  20.3]
20
[[  1.81000000e+01   1.85729380e+03   1.32386860e+03   4.11300000e+00
    2.68800000e+01   2.60000000e-01   1.10240981e+00   2.27119566e+01]
 [  1.82000000e+01   1.94891430e+03   1.36043460e+03   4.11300000e+00
    2.73700000e+01   2.30000000e-01   1.10240981e+00   8.39679178e+01]]
[ 18.1  18.2]
2


In [40]:
pairs1=np.array(list(itertools.combinations(data1[:,0],2)))
sys_diff=pairs1[:,1].astype(int)-pairs1[:,0].astype(int)
img_diff=pairs1[:,1]-pairs1[:,0]
pairs1=np.array(pairs1[np.logical_and(sys_diff==0, img_diff<>0)])
print(pairs1)
pairs2=np.array(list(itertools.combinations(data2[:,0],2)))
diff=pairs2[:,1].astype(int)-pairs2[:,0].astype(int)
pairs2=np.array(pairs2[np.where(diff==0)])
print(pairs2)

[[  4.2   4.3]
 [  6.1   6.2]
 [  6.1   6.3]
 [  6.2   6.3]
 [  7.1   7.3]
 [  8.1   8.2]
 [  8.1   8.3]
 [  8.2   8.3]
 [ 11.1  11.2]
 [ 11.1  11.3]
 [ 11.2  11.3]
 [ 12.1  12.2]
 [ 12.1  12.3]
 [ 12.1  12.4]
 [ 12.2  12.3]
 [ 12.2  12.4]
 [ 12.3  12.4]
 [ 20.1  20.2]
 [ 20.1  20.3]
 [ 20.2  20.3]]
[[ 18.1  18.2]]


In [41]:
mag_src1=np.array([float(data1[np.where(data1[:,0]==pairs1[x,0]),4]) for x in range(len(pairs1))])
#print(mag_src1)
mag_obs1=np.array([float(data1[np.where(data1[:,0]==pairs1[x,1]),4]) for x in range(len(pairs1))])
print(mag_obs1)
M_src1=np.array([float(data1[np.where(data1[:,0]==pairs1[x,0]),7]) for x in range(len(pairs1))])
#print(M_src1)
M_obs1=np.array([float(data1[np.where(data1[:,0]==pairs1[x,1]),7]) for x in range(len(pairs1))])
#print(M_obs1)
mag_err_src1=np.array([float(data1[np.where(data1[:,0]==pairs1[x,0]),5]) for x in range(len(pairs1))])
mag_err_obs1=np.array([float(data1[np.where(data1[:,0]==pairs1[x,1]),5]) for x in range(len(pairs1))])
print(mag_err_obs1)

mag_pre1=[mag_src1[x]-2.5*math.log10(M_obs1[x]/M_src1[x]) for x in range(len(pairs1))]
print (mag_pre1)
print (mag_pre1-mag_obs1)
print (max(mag_pre1))
print (min(mag_pre1))
print (max(mag_obs1))
print (min(mag_obs1))



mag_src2=np.array([float(data2[np.where(data2[:,0]==pairs2[x,0]),4]) for x in range(len(pairs2))])
mag_obs2=np.array([float(data2[np.where(data2[:,0]==pairs2[x,1]),4]) for x in range(len(pairs2))])
M_src2=np.array([float(data2[np.where(data2[:,0]==pairs2[x,0]),7]) for x in range(len(pairs2))])
M_obs2=np.array([float(data2[np.where(data2[:,0]==pairs2[x,1]),7]) for x in range(len(pairs2))])
mag_err_src2=np.array([float(data2[np.where(data2[:,0]==pairs2[x,0]),5]) for x in range(len(pairs2))])
mag_err_obs2=np.array([float(data2[np.where(data2[:,0]==pairs2[x,1]),5]) for x in range(len(pairs2))])
mag_pre2=[mag_src2[x]-2.5*math.log10(M_obs2[x]/M_src2[x]) for x in range(len(pairs2))]
print (mag_pre2-mag_obs2)

[ 23.81  22.19  22.51  22.51  24.05  24.55  24.71  24.71  25.54  27.26
  27.26  25.47  25.11  27.85  25.11  27.85  27.85  25.94  26.13  26.13]
[ 0.03  0.02  0.01  0.01  0.04  0.04  0.04  0.04  0.06  0.2   0.2   0.24
  0.07  0.6   0.07  0.6   0.6   0.07  0.13  0.13]
[21.928368721327622, 20.876613918354668, 21.354589367618491, 22.667975449263825, 23.573164151626781, 23.938568535518318, 24.289581170142593, 24.901012634624273, 26.691382112502499, 27.553662933438144, 26.402280820935644, 25.15182048921606, 26.56625198768727, 26.975962862607531, 26.884431498471208, 27.29414237339147, 25.519710874920261, 25.95569720607217, 26.670467762271599, 26.654770556199431]
[-1.88163128 -1.31338608 -1.15541063  0.15797545 -0.47683585 -0.61143146
 -0.42041883  0.19101263  1.15138211  0.29366293 -0.85771918 -0.31817951
  1.45625199 -0.87403714  1.7744315  -0.55585763 -2.33028913  0.01569721
  0.54046776  0.52477056]
27.5536629334
20.8766139184
27.85
22.19
[-1.90964709]


In [42]:
x1=mag_obs1
y1=mag_pre1
xerr1=mag_err_obs1+0.2
print(xerr1)
yerr1=mag_err_src1+0.2
x2=mag_obs2
y2=mag_pre2
xerr2=mag_err_obs2+0.2
yerr2=mag_err_src2+0.2
x=range(18,30)
#print(x1-y1)
#print(x2-y2)



fig=plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)
ax1.set_xlabel("observed magnitude",fontsize=12)
ax1.set_ylabel("predicted magnitude",fontsize=12)
ax1.set_xlim(20,29)
ax1.set_ylim(20,29)
ax1.grid(True,linestyle='-',color='0.75')

ax1.scatter(x1,y1,s=60,alpha=0.7, color='#0000CC', marker="v", label='In Model') 
#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(x1,y1)
#print slope1
#print 'r value', r_value1
#print  'p_value', p_value1
#print 'standard deviation', std_err1
#line1 = slope1*x+intercept1
#ax1.plot(x,line1)
#print (line1)

fit1 = np.polyfit(x1,y1,1)
fit_fn1 = np.poly1d(fit1)
ax1.plot(x, fit_fn1(x), color='#0000CC')
print ("y=%.6fx+(%.6f)" %(fit1[0],fit1[1]))
plt.errorbar(x1,y1, yerr=yerr1, xerr=xerr1, linestyle="None", ecolor="#888888")

ax1.scatter(x2,y2,s=30,alpha=0.9, color='#FF5000', marker="D", label='Candidate')
#slope, intercept, r_value, p_value, std_err = stats.linregress(x2,y2)
#print slope
#fit2 = np.polyfit(x2,y2,1)
#fit_fn2 = np.poly1d(fit2)
#ax1.plot(x, fit_fn2(x), color='#FF5000')
#print ("y=%.6fx+(%.6f)" %(fit2[0],fit2[1]))
plt.errorbar(x2,y2,yerr=yerr2,xerr=xerr2, linestyle="None", ecolor="#888888")

ax1.plot(x, x, color='#EEEE00',linestyle='--', linewidth=3)

plt.legend(loc='upper left');
plt.show()

[ 0.23  0.22  0.21  0.21  0.24  0.24  0.24  0.24  0.26  0.4   0.4   0.44
  0.27  0.8   0.27  0.8   0.8   0.27  0.33  0.33]
y=1.013404x+(-0.573973)


In [43]:
m_ratio1=[math.log10(M_src1[x]/M_obs1[x]) for x in range(len(pairs1))]
m_ratio2=[math.log10(M_src2[x]/M_obs2[x]) for x in range(len(pairs2))]

x1=m_ratio1
y1=(mag_src1-mag_obs1)/(-2.5)
xerr1=(mag_err_obs1+mag_err_src1)/(-2.5)+0.2
print(xerr1)
yerr1=0.2
x2=m_ratio2
y2=(mag_src2-mag_obs2)/(-2.5)
xerr2=(mag_err_obs2+mag_err_src2)/(-2.5)+0.2
yerr2=0.2
x=range(-2,5)
#print(x1-y1)
#print(x2-y2)



fig=plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)
ax1.set_xlabel("log(magnification ratio)",fontsize=12)
ax1.set_ylabel("log(flux ratio)",fontsize=12)
ax1.set_xlim(-1,1.5)
ax1.set_ylim(-1,1.5)
ax1.grid(True,linestyle='-',color='0.75')

ax1.scatter(x1,y1,s=60,alpha=0.7, color='#0000CC', marker="v", label='In Model') 
#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(x1,y1)
#print slope1
#print 'r value', r_value1
#print  'p_value', p_value1
#print 'standard deviation', std_err1
#line1 = slope1*x+intercept1
#ax1.plot(x,line1)
#print (line1)

fit1 = np.polyfit(x1,y1,1)
fit_fn1 = np.poly1d(fit1)
ax1.plot(x, fit_fn1(x), color='#0000CC')
print ("y=%.6fx+(%.6f)" %(fit1[0],fit1[1]))
plt.errorbar(x1,y1, yerr=yerr1, xerr=xerr1, linestyle="None", ecolor="#888888")

ax1.scatter(x2,y2,s=30,alpha=0.9, color='#FF5000', marker="D", label='Candidate')
#slope, intercept, r_value, p_value, std_err = stats.linregress(x2,y2)
#print slope

fit2 = np.polyfit(x2,y2,1)
fit_fn2 = np.poly1d(fit2)
ax1.plot(x, fit_fn2(x), color='#FF5000')
print ("y=%.6fx+(%.6f)" %(fit2[0],fit2[1]))
plt.errorbar(x2,y2,yerr=yerr2,xerr=xerr2, linestyle="None", ecolor="#888888")

ax1.plot(x, x, color='#EEEE00',linestyle='--', linewidth=3)

plt.legend(loc='upper left');
plt.show()

[ 0.172  0.188  0.192  0.188  0.168  0.18   0.18   0.168  0.144  0.088
  0.096  0.072  0.14  -0.072  0.076 -0.136 -0.068  0.144  0.12   0.12 ]
y=0.421227x+(0.196702)
y=-0.172578x+(0.098000)


/home/jess/anaconda2/lib/python2.7/site-packages/numpy/lib/polynomial.py:594: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
